# HVAC Climate Data Pipeline - Exploration & Visualization

This notebook demonstrates the analysis and visualization of HVAC comfort metrics from the data pipeline.

**Features:**
- Indoor vs outdoor temperature comparison
- Overcooling detection and visualization
- Stale air (high CO₂) analysis
- Per-room comfort metrics

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
from datetime import datetime, timedelta

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 1. Load Data from Silver Layer

Load the comfort facts from the silver layer which includes:
- Indoor sensor readings (temperature, humidity, CO₂, VOC)
- Outdoor weather data
- Computed comfort flags (overcooled, stale air)

In [ ]:
# Define data paths
BASE_PATH = Path('..')
SILVER_PATH = BASE_PATH / 'data' / 'silver' / 'hvac_comfort_facts'
GOLD_PATH = BASE_PATH / 'data' / 'gold' / 'daily_comfort_metrics'

# Load silver layer comfort facts for analysis
# For demo, we'll load data for 2025-01-15
target_date = '2025-01-15'
date_obj = datetime.strptime(target_date, '%Y-%m-%d')

silver_partition = SILVER_PATH / f'year={date_obj.year}' / f'month={date_obj.month:02d}' / f'day={date_obj.day:02d}'

if silver_partition.exists():
    comfort_df = pd.read_parquet(silver_partition)
    print(f"Loaded {len(comfort_df)} records from silver layer")
    print(f"\nColumns: {comfort_df.columns.tolist()}")
    print(f"\nData shape: {comfort_df.shape}")
    comfort_df.head()
else:
    print(f"Error: Data not found at {silver_partition}")
    print("Please run the pipeline first: python scripts/ingest_transform.py --date 2025-01-15")

## 2. Data Overview & Summary Statistics

In [ ]:
# Summary statistics
print("=" * 60)
print("Summary Statistics")
print("=" * 60)

print(f"\nTotal readings: {len(comfort_df)}")
print(f"Unique rooms: {comfort_df['room_id'].nunique()}")
print(f"Room IDs: {sorted(comfort_df['room_id'].unique())}")

print(f"\nTime range: {comfort_df['ts_utc'].min()} to {comfort_df['ts_utc'].max()}")

print("\n" + "=" * 60)
print("Temperature Statistics")
print("=" * 60)
print(comfort_df[['indoor_temp_c', 'outdoor_temp_c']].describe())

print("\n" + "=" * 60)
print("Air Quality Statistics")
print("=" * 60)
print(comfort_df[['indoor_co2_ppm', 'indoor_voc_ppb']].describe())

## 3. Indoor vs Outdoor Temperature Comparison

Visualize how indoor temperatures compare to outdoor conditions across all rooms.

In [ ]:
# Create figure with multiple traces
fig = go.Figure()

# Add outdoor temperature as a reference line
fig.add_trace(go.Scatter(
    x=comfort_df['ts_utc'],
    y=comfort_df['outdoor_temp_c'],
    mode='lines',
    name='Outdoor Temperature',
    line=dict(color='blue', width=3, dash='dash')
))

# Add indoor temperature for each room
colors = ['red', 'green', 'orange', 'purple']
for idx, room in enumerate(sorted(comfort_df['room_id'].unique())):
    room_data = comfort_df[comfort_df['room_id'] == room]
    fig.add_trace(go.Scatter(
        x=room_data['ts_utc'],
        y=room_data['indoor_temp_c'],
        mode='lines',
        name=f'Indoor - {room}',
        line=dict(color=colors[idx], width=2)
    ))

fig.update_layout(
    title='Indoor vs Outdoor Temperature Over Time',
    xaxis_title='Time',
    yaxis_title='Temperature (°C)',
    hovermode='x unified',
    height=500,
    legend=dict(x=1.05, y=1)
)

fig.show()

## 4. Overcooling Detection & Visualization

Highlight periods where rooms were overcooled:
- **Overcooled** = Indoor temp < 21°C AND Outdoor temp > 25°C

In [ ]:
# Count overcooling events
print("=" * 60)
print("Overcooling Analysis")
print("=" * 60)

overcooled_counts = comfort_df.groupby('room_id')['overcooled_flag'].sum().sort_values(ascending=False)
total_readings = comfort_df.groupby('room_id')['overcooled_flag'].count()
overcooled_pct = (overcooled_counts / total_readings * 100).round(2)

print("\nOvercooling events by room:")
for room in overcooled_counts.index:
    print(f"  {room}: {overcooled_counts[room]} events ({overcooled_pct[room]}% of time)")

print(f"\nTotal overcooling events: {comfort_df['overcooled_flag'].sum()}")

In [ ]:
# Visualize temperature with overcooling periods highlighted
fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=[f'Room: {room}' for room in sorted(comfort_df['room_id'].unique())],
    vertical_spacing=0.05
)

for idx, room in enumerate(sorted(comfort_df['room_id'].unique()), 1):
    room_data = comfort_df[comfort_df['room_id'] == room].copy()
    
    # Indoor temperature line
    fig.add_trace(
        go.Scatter(
            x=room_data['ts_utc'],
            y=room_data['indoor_temp_c'],
            mode='lines',
            name=f'{room} - Indoor',
            line=dict(color='blue'),
            showlegend=(idx == 1)
        ),
        row=idx, col=1
    )
    
    # Highlight overcooled periods
    overcooled_data = room_data[room_data['overcooled_flag'] == True]
    if not overcooled_data.empty:
        fig.add_trace(
            go.Scatter(
                x=overcooled_data['ts_utc'],
                y=overcooled_data['indoor_temp_c'],
                mode='markers',
                name='Overcooled',
                marker=dict(color='red', size=8, symbol='x'),
                showlegend=(idx == 1)
            ),
            row=idx, col=1
        )

fig.update_layout(
    title_text='Indoor Temperature with Overcooling Periods Highlighted',
    height=1000,
    showlegend=True
)

fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Temperature (°C)')

fig.show()

## 5. Percentage Time Overcooled - Bar Chart

In [ ]:
# Load gold layer for aggregated metrics
gold_partition = GOLD_PATH / f'year={date_obj.year}' / f'month={date_obj.month:02d}' / f'day={date_obj.day:02d}'

if gold_partition.exists():
    gold_df = pd.read_parquet(gold_partition)
    
    # Create bar chart
    fig = px.bar(
        gold_df,
        x='room_id',
        y='pct_time_overcooled',
        title='Percentage of Time Overcooled by Room',
        labels={'room_id': 'Room', 'pct_time_overcooled': '% Time Overcooled'},
        color='pct_time_overcooled',
        color_continuous_scale='Reds',
        text='pct_time_overcooled'
    )
    
    fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
    fig.update_layout(height=400, showlegend=False)
    fig.show()
else:
    print(f"Gold layer data not found at {gold_partition}")

## 6. Air Quality Analysis - CO₂ Levels

Analyze CO₂ levels to detect stale air conditions (CO₂ > 1000 ppm)

In [ ]:
# Stale air analysis
print("=" * 60)
print("Air Quality Analysis - CO₂ Levels")
print("=" * 60)

stale_air_counts = comfort_df.groupby('room_id')['stale_air_flag'].sum().sort_values(ascending=False)
total_readings = comfort_df.groupby('room_id')['stale_air_flag'].count()
stale_air_pct = (stale_air_counts / total_readings * 100).round(2)

print("\nStale air events by room:")
for room in stale_air_counts.index:
    print(f"  {room}: {stale_air_counts[room]} events ({stale_air_pct[room]}% of time)")

print(f"\nTotal stale air events: {comfort_df['stale_air_flag'].sum()}")

# Average CO2 by room
avg_co2 = comfort_df.groupby('room_id')['indoor_co2_ppm'].mean().sort_values(ascending=False)
print("\nAverage CO₂ levels by room:")
for room in avg_co2.index:
    print(f"  {room}: {avg_co2[room]:.1f} ppm")

In [ ]:
# CO2 time series plot
fig = go.Figure()

# Add threshold line
fig.add_hline(y=1000, line_dash="dash", line_color="red", 
              annotation_text="Stale Air Threshold (1000 ppm)")

# Add CO2 levels for each room
for room in sorted(comfort_df['room_id'].unique()):
    room_data = comfort_df[comfort_df['room_id'] == room]
    fig.add_trace(go.Scatter(
        x=room_data['ts_utc'],
        y=room_data['indoor_co2_ppm'],
        mode='lines',
        name=room
    ))

fig.update_layout(
    title='Indoor CO₂ Levels Over Time',
    xaxis_title='Time',
    yaxis_title='CO₂ Concentration (ppm)',
    hovermode='x unified',
    height=500
)

fig.show()

## 7. Combined Comfort Dashboard

In [ ]:
if gold_partition.exists():
    # Create a comprehensive dashboard
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            '% Time Overcooled by Room',
            '% Time with Stale Air by Room',
            'Average Indoor Temperature',
            'Average CO₂ Levels'
        ),
        specs=[[{'type': 'bar'}, {'type': 'bar'}],
               [{'type': 'bar'}, {'type': 'bar'}]]
    )
    
    # Plot 1: % Time Overcooled
    fig.add_trace(
        go.Bar(x=gold_df['room_id'], y=gold_df['pct_time_overcooled'], 
               name='% Overcooled', marker_color='red'),
        row=1, col=1
    )
    
    # Plot 2: % Time Stale Air
    fig.add_trace(
        go.Bar(x=gold_df['room_id'], y=gold_df['pct_time_stale_air'], 
               name='% Stale Air', marker_color='orange'),
        row=1, col=2
    )
    
    # Plot 3: Avg Indoor Temperature
    fig.add_trace(
        go.Bar(x=gold_df['room_id'], y=gold_df['avg_indoor_temp'], 
               name='Avg Temp', marker_color='blue'),
        row=2, col=1
    )
    
    # Plot 4: Avg CO2
    fig.add_trace(
        go.Bar(x=gold_df['room_id'], y=gold_df['avg_indoor_co2'], 
               name='Avg CO₂', marker_color='green'),
        row=2, col=2
    )
    
    fig.update_xaxes(title_text='Room', row=1, col=1)
    fig.update_xaxes(title_text='Room', row=1, col=2)
    fig.update_xaxes(title_text='Room', row=2, col=1)
    fig.update_xaxes(title_text='Room', row=2, col=2)
    
    fig.update_yaxes(title_text='%', row=1, col=1)
    fig.update_yaxes(title_text='%', row=1, col=2)
    fig.update_yaxes(title_text='°C', row=2, col=1)
    fig.update_yaxes(title_text='ppm', row=2, col=2)
    
    fig.update_layout(
        title_text=f'HVAC Comfort Metrics Dashboard - {target_date}',
        height=800,
        showlegend=False
    )
    
    fig.show()

## 8. Key Findings & Recommendations

Based on the analysis above:

1. **Temperature Management**: Monitor rooms showing wide variations in temperature
2. **Overcooling**: Investigate any rooms with >5% time overcooled
3. **Air Quality**: Address rooms where CO₂ frequently exceeds 1000 ppm
4. **Energy Efficiency**: Overcooling wastes energy - optimize HVAC setpoints

### Next Steps:
- Set up automated alerts for comfort violations
- Analyze trends over longer time periods
- Correlate with occupancy data
- Implement predictive maintenance based on patterns